# CARIA Architecture: The Physics Lab (Empirical Validation)

## Hypothesis: Market Density ($\rho$)
We posit that the "Regime" of an asset is defined by its response to Synchronization Pressure.
- **Heavy Asset ($\rho < 0$)**: obeys Gravity. Pressure (Sync) causes Collapse. (e.g., S&P 500)
- **Light Asset ($\rho > 0$)**: obeys Levitation. Pressure (Sync) causes Bubbles. (e.g., Bitcoin)
- **Inert Asset ($\rho \approx 0$)**: ignores Pressure. (e.g., Gold)

This notebook uses **REAL MARKET DATA** to empirically prove these physical laws.

In [ ]:
!pip install yfinance PyWavelets scikit-learn numpy pandas scipy matplotlib -q
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import yfinance as yf

# ==========================================
# 1. REAL DATA LOADER
# ==========================================
def fetch_data(ticker, start='2010-01-01'):
    print(f"Fetching {ticker}...")
    df = yf.download(ticker, start=start, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df = df.xs('Close', axis=1, level=0)
    return df.iloc[:, 0].dropna() if isinstance(df, pd.DataFrame) else df.dropna()

sp500 = fetch_data('^GSPC')
btc = fetch_data('BTC-USD')
tlt = fetch_data('TLT')
gold = fetch_data('GC=F')

assets = {
    'S&P 500 (Equity)': sp500,
    'Bitcoin (Crypto)': btc,
    'TLT (Bonds)': tlt,
    'Gold (Commodity)': gold
}

print("Market Data Ready for Physics Analysis.")

# ==========================================
# 2. MOTOR FÍSICO GREAT CARIA (v5.1)
# ==========================================
def calculate_metrics(price_series):
    ret = price_series.pct_change().dropna()
    
    # A. Descomposición Temporal (Relatividad)
    # Bandas de frecuencia: Rápida (Ruido), Media (Resonancia), Lenta (Estructura)
    fast = ret - ret.rolling(5).mean()
    med = ret.rolling(5).mean() - ret.rolling(30).mean()
    slow = ret.rolling(30).mean()
    
    # Limpieza de datos para Hilbert
    fast = fast.fillna(0)
    med = med.fillna(0)
    slow = slow.fillna(0)
    
    # B. Transformada de Hilbert (Fase Instantánea)
    # Al usar retornos reales, aseguramos que la fase capture la direccionalidad
    phases = np.angle(np.column_stack([
        signal.hilbert(fast.values), 
        signal.hilbert(med.values), 
        signal.hilbert(slow.values)
    ]))
    
    # C. Sincronización de Kuramoto (La Señal Universal)
    # r va de 0 (Caos) a 1 (Orden Total)
    r = np.abs(np.mean(np.exp(1j * phases), axis=1))
    sync = pd.Series(r, index=ret.index).rolling(20).mean().fillna(0)
    
    # D. Retornos Futuros (Para medir la Reacción del Activo)
    fwd_ret = ret.shift(-20).rolling(5).mean() # Reacción a 1 mes (20 días)
    
    return sync, fwd_ret, ret

# ==========================================
# 3. ANÁLISIS DE DENSIDAD Y ESTRATEGIA
# ==========================================
results = {}

print("Running Physics Engine on Real Data...")

for name, price in assets.items():
    if price.empty: continue
    sync, fwd_ret, ret = calculate_metrics(price)
    
    # --- CÁLCULO DE DENSIDAD EMPÍRICA ---
    # Densidad = Correlación(Presión, Reacción)
    density = sync.rolling(252).corr(fwd_ret).fillna(0)
    avg_density = density.mean()
    
    # --- SELECTOR DE ESTRATEGIA (Universal Switch) ---
    threshold = 0.8
    is_fragile = sync > threshold
    
    # Usamos la densidad empírica media para clasificar el activo
    if avg_density < -0.01: 
        # S&P 500 debería caer aquí (Densidad Negativa)
        signal_arr = np.where(is_fragile, 0, 1) 
        strategy_type = "Heavy (Sell on Sync)"
        color = 'red'
    elif avg_density > 0.01: 
        # Bitcoin debería caer aquí (Densidad Positiva/Burbuja)
        signal_arr = np.ones(len(ret)) 
        strategy_type = "Light (Ride Sync)"
        color = 'green'
    else:
        # Gold/TLT deberían caer aquí (Densidad Neutra)
        signal_arr = np.ones(len(ret))
        strategy_type = "Neutral (Hold)"
        color = 'gray'
        
    # Backtest
    strat_ret = ret * pd.Series(signal_arr, index=ret.index).shift(1).fillna(1)
    cum_bh = (1 + ret).cumprod()
    cum_strat = (1 + strat_ret).cumprod()
    
    # Align indices for plotting
    common_idx = sync.index.intersection(fwd_ret.index)
    
    results[name] = {
        'Sync': sync.loc[common_idx],
        'Returns': fwd_ret.loc[common_idx],
        'Avg_Density': avg_density,
        'Strategy_Type': strategy_type,
        'Cum_BH': cum_bh,
        'Cum_Strat': cum_strat,
        'Color': color
    }
    
    print(f" > {name}: Densidad Media = {avg_density:.4f} -> Régimen: {strategy_type}")

# ==========================================
# 4. VISUALIZACIÓN DE LA FÍSICA REAL
# ==========================================

# Gráfico A: La Densidad (Scatter)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
fig.suptitle('A. Market Physics: Empirical Density Analysis (Real Data)', fontsize=16)

for i, (name, res) in enumerate(results.items()):
    if i >= 4: break
    ax = axes[i]
    # Scatter plot
    x = res['Sync']
    y = res['Returns'] * 100
    
    # Subsample for speed/clarity if needed
    if len(x) > 2000:
        mask = np.random.choice([True, False], len(x), p=[0.2, 0.8])
        x, y = x[mask], y[mask]

    valid = ~np.isnan(x) & ~np.isnan(y)
    x, y = x[valid], y[valid]

    ax.scatter(x, y, alpha=0.1, c=res['Color'], label=f"Density: {res['Avg_Density']:.3f}")
    
    # Regresión
    if len(x) > 0:
        m, b = np.polyfit(x, y, 1)
        ax.plot(x, m*x + b, color='black', linewidth=2, linestyle='--', label='Vector')
    
    ax.set_title(f"{name}")
    ax.set_xlabel("Sync (Pressure)")
    ax.set_ylabel("Future Returns (Reaction)")
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Gráfico B: Performance
fig2, axes2 = plt.subplots(2, 2, figsize=(15, 10))
axes2 = axes2.flatten()
fig2.suptitle('B. Physics-Based Strategy Performance', fontsize=16)

for i, (name, res) in enumerate(results.items()):
    if i >= 4: break
    ax = axes2[i]
    ax.plot(res['Cum_BH'], label='Buy & Hold', color='gray', alpha=0.5)
    ax.plot(res['Cum_Strat'], label=f"Smart ({res['Strategy_Type']})", color=res['Color'], linewidth=2)
    ax.set_title(f"{name}")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()